# Libraries

In [ ]:
# Installazione delle librerie necessarie con specifiche per l'uso della GPU
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install openpyxl
!pip install pandas

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.9 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-69.5.1-py3-none-any.whl (894 kB)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 1.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 27.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.8 MB/s eta 0:00:00
    Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-2jxgzz6g/overlay/local/bin
    changing mode of /tmp/pip-build-env-2jxgzz6g/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-2jxgzz6g/overlay/local/bin/wheel to 755
    changing mode of /tmp/pip

In [ ]:
# Importing necessary libraries
import spacy
import json
import os
import re
import csv
import openpyxl
import pandas as pd
import numpy as np
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Corpus Analysis

In [ ]:
# Load the Excel file
data_path = '/content/drive/My Drive/Llama2_Translation_LNC/interaction-corpus.xlsx'
data = pd.read_excel(data_path)

# Display the first few rows of the dataset to understand its structure
print(data.head())

                            CODE ID  Turn Participant  \
0  ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa     1           U   
1  ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa     2           S   
2  ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa     3           U   
3  ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa     4           S   
4  ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa     5           U   

                                                Text       Version Experiment  \
0                                                hi   [2023-05-28]       [Si]   
1  I don't know, try asking something like, "What...  [2023-05-28]       [Si]   
2                            describe the automaton   [2023-05-28]       [Si]   
3  It is an automaton which accepts zero or more ...  [2023-05-28]       [Si]   
4           There is a transition between q2 and q0   [2023-05-28]       [Si]   

  VIP  Token count                 Errors_GOLD  
0  no            1                         NaN  
1  no           13                Topic change  
2  no            3  Ign

In [ ]:
# Filter the data for system responses only
system_responses = data[data['Participant'] == 'S']

# Count the types of errors for system responses
system_error_counts = system_responses['Errors_GOLD'].value_counts()
system_error_counts

Errors_GOLD
Topic change                                116
Straight wrong response                      28
Indirect response                            16
Excess of information                        13
Lack of information                           9
Indirect response; Topic change               3
Excess of information; Indirect response      1
Ignoring question/feedback                    1
Name: count, dtype: int64

In [ ]:
# Filter the data for system responses only
user_responses = data[data['Participant'] == 'U']

# Count the types of errors for system responses
system_error_counts = user_responses['Errors_GOLD'].value_counts()
system_error_counts

Errors_GOLD
Repetition                                        43
Ignoring question/feedback                        21
Grammatical error                                 18
Ignoring question/feedback; Repetition            13
Off-topic                                          4
Non-understandable                                 4
Lack of information                                4
Non-cooperativity                                  3
Ignoring question/feedback; Non-understandable     2
Ill-formed                                         2
Ignoring question/feedback; Off-topic              1
Name: count, dtype: int64

Escludiamo:
1. Off-topic
3. Non-understandable
4. Non-cooperativity
5. Repetition

## Topic Change

In [ ]:
# Identify rows where the system response has a "Topic change" error
topic_change_indices = system_responses[system_responses['Errors_GOLD'] == 'Topic change'].index

# Get the corresponding user inputs just before the system responses with "Topic Change"
user_inputs_before_topic_change = data.loc[topic_change_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_tc = user_inputs_before_topic_change[
    user_inputs_before_topic_change['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_topic_change['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_tc = filtered_user_inputs_tc['Text'].sample(15).tolist()
final_user_queries_tc

['Talk me about transitions\xa0',
 'is there any self loop',
 'What transitions does the automaton have\xa0',
 'how many 0 art here\xa0',
 'What is an example of accepted string\xa0',
 'What could be a minimal spatial representation for this automaton\xa0',
 'Summarise the automaton\xa0',
 'What are the accepted inputs\xa0',
 'what is the regular expression describing the automata\xa0',
 'How can I define an automaton\xa0',
 'hi\xa0',
 'what is the input',
 'There is a transition between q5 and q7\xa0',
 'Does it only accept 1s and 0s\xa0',
 'what is the initial state of the automaton\xa0']

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_tc)
print("La lunghezza dell'array è:", array_length)

La lunghezza dell'array è: 75


## Straight Wrong Response

In [ ]:
# Identify rows where the system response has a "Straight wrong response" error
straight_wrong_response_indices = system_responses[system_responses['Errors_GOLD'] == 'Straight wrong response'].index

# Get the corresponding user inputs just before the system responses with "Straight wrong response"
user_inputs_before_straight_wrong_response = data.loc[straight_wrong_response_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_swr = user_inputs_before_straight_wrong_response[
    user_inputs_before_straight_wrong_response['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_straight_wrong_response['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_swr = filtered_user_inputs_swr['Text'].sample(10).tolist()
final_user_queries_swr

['How many states does a pentagon shaped automaton have\xa0',
 'arc from q2 to q1',
 'how many arcs marked by 1 are there\xa0',
 'q2',
 'Which transitions end in q0\xa0',
 'Is there an arc between q0 and q1\xa0',
 'If q2 is the final state the language accepted by the automaton is the same\xa0',
 'transitions',
 'Is there an arc between q1 and q2\xa0',
 'q0']

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_swr)
print("La lunghezza dell'array è:", array_length)

La lunghezza dell'array è: 25


In [ ]:
filtered_user_inputs_swr['Text'].tolist()

['What are the states\xa0',
 'If q2 is the final state the language accepted by the automaton is the same\xa0',
 'is q4 linked to q0\xa0',
 'Is there an arc from q4 to q0\xa0',
 'If q2 were the final state the language accepted by the automaton is the same\xa0',
 'How many states can I use in an automaton\xa0',
 'Which transitions end in q0\xa0',
 'is there an arc between q0 and q1\xa0',
 'How many states does a pentagon shaped automaton have\xa0',
 'Is there an arc between q0 and q1\xa0',
 'Is there an arc between q1 and q2\xa0',
 'is there an arc from q2 to q1\xa0',
 'If q2 were the final state the language accepted by the automaton is the same\xa0',
 'If q0 were the final state the language accepted by the automaton is the same\xa0',
 'transitions',
 'arcs between q4 and q0',
 'arc from q2 to q1',
 'how many arcs marked by 1 are there\xa0',
 'describe the shape of the automaton',
 'does the automaton accept other languages',
 'q0',
 'q2',
 'q3 to q2',
 'q4 to q0',
 'describe the pos

## Indirect Response

In [ ]:
# Identify rows where the system response has a "Indirect response" error
indirect_response_indices = system_responses[system_responses['Errors_GOLD'] == 'Indirect response'].index

# Get the corresponding user inputs just before the system responses with "Indirect response"
user_inputs_before_indirect_response = data.loc[indirect_response_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_ir = user_inputs_before_indirect_response[
    user_inputs_before_indirect_response['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_indirect_response['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_ir = filtered_user_inputs_ir['Text'].sample(8).tolist()
final_user_queries_ir

['What happens when a 0 comes to state q1\xa0',
 'The automaton accepts a language allowing words made of an odd number of 0s and 1s\xa0',
 'is there an arc from q0 to q1\xa0',
 'q0 to q4',
 'Is 11100 part of the accepted language\xa0',
 'The automaton accepts a language allowing words made of an odd number of 0s and 1s\xa0',
 'is q0 linked to q0\xa0',
 'Is there a transition from q0 to q5\xa0']

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_ir)
print("La lunghezza dell'array è:", array_length)

La lunghezza dell'array è: 16


In [ ]:
filtered_user_inputs_ir['Text'].tolist()

['The automaton accepts a language allowing words made of an odd number of 0s and 1s\xa0',
 'What happens when a 0 comes to state q1\xa0',
 'is q0 linked to q0\xa0',
 'is q1 linked to q4\xa0',
 'Is 11100 part of the accepted language\xa0',
 'Is there a transition from q0 to q5\xa0',
 'is q4 a final state\xa0',
 'Is there a transition from q0 to q5\xa0',
 'The automaton accepts a language allowing words made of an odd number of 0s and 1s\xa0',
 'Is there a transition from q0 to q5\xa0',
 'The automaton accepts a language allowing words made of an odd number of 0s and 1s\xa0',
 'is there an arc from q0 to q1\xa0',
 'The automaton accepts a language allowing words made of an odd number of 0s and 1s\xa0',
 'Does the automaton accept a language allowing words made of an odd number of 0s and 1s\xa0',
 'how many transitions',
 'q0 to q4']

## Excess of Information

In [ ]:
# Identify rows where the system response has a "Excess of information" error
excess_of_information_indices = system_responses[system_responses['Errors_GOLD'] == 'Excess of information'].index

# Get the corresponding user inputs just before the system responses with "Excess of information"
user_inputs_before_excess_of_information = data.loc[excess_of_information_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_eoi = user_inputs_before_excess_of_information[
    user_inputs_before_excess_of_information['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_excess_of_information['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_eoi = filtered_user_inputs_eoi['Text'].sample(4).tolist()
final_user_queries_eoi

['How many transitions start from q0\xa0',
 'Please describe the transitions\xa0',
 'describe the arcs',
 'can you describe the transitions']

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_eoi)
print("La lunghezza dell'array è:", array_length)

La lunghezza dell'array è: 9


In [ ]:
filtered_user_inputs_eoi['Text'].tolist()

['can you describe the transitions\xa0',
 'Please describe the transitions\xa0',
 'How many transitions start from q0\xa0',
 'how many transitions\xa0',
 'how is marked the arc between q0 and q2\xa0',
 'which is final state',
 'label from q3 to q4',
 'describe the arcs',
 'can you describe the transitions']

## Lack of Information

In [ ]:
# Identify rows where the system response has a "Lack of information" error
lack_of_information_indices = system_responses[system_responses['Errors_GOLD'] == 'Lack of information'].index

# Get the corresponding user inputs just before the system responses with "Lack of information"
user_inputs_before_lack_of_information = data.loc[lack_of_information_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_loi = user_inputs_before_lack_of_information[
    user_inputs_before_lack_of_information['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_lack_of_information['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_loi = filtered_user_inputs_loi['Text'].sample(4).tolist()
final_user_queries_loi

['What transitions enter and exit q4\xa0',
 'are there arcs between q2 and q0\xa0',
 'What transitions enter and exit q2\xa0',
 'Briefly describe the automaton How many states are there\xa0']

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_loi)
print("La lunghezza dell'array è:", array_length)

La lunghezza dell'array è: 9


In [ ]:
filtered_user_inputs_loi['Text'].tolist()

['There is a transition between q2 and q0\xa0',
 'What transitions enter and exit q0\xa0',
 'What transitions enter and exit q1\xa0',
 'What transitions enter and exit q2\xa0',
 'What transitions enter and exit q4\xa0',
 'What transitions enter and exit q3\xa0',
 'are there arcs between q2 and q0\xa0',
 'Briefly describe the automaton How many states are there\xa0',
 'Is there a transition from q0 to q5\xa0']

# Chiamata al modello

In [ ]:
# Seleziono LLama2 13B come modello
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

# Download del modello da Hugging Face Hub
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# Inizializzazione del modello con configurazione per GPU
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # Numero di core della CPU
    n_batch=512,  # Numero di batch, dipende dalla VRAM della GPU
    n_gpu_layers=32  # Numero di layer gestiti dalla GPU
)
lcpp_llm.params.n_gpu_layers

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


32

In [ ]:
# Generazione della risposta
def call_model(prompt):
  response = lcpp_llm(prompt=prompt, max_tokens=4096)
  #print(response["choices"][0]["text"])
  return response["choices"][0]["text"]

# Salvataggio Dati

In [ ]:
def setup_xlsx(file_path):
    # Controlla se il file esiste
    if not os.path.exists(file_path):
        # Crea un nuovo workbook e aggiunge un foglio
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = "Data"
        ws.append(['Input', 'Translation', 'Output', 'Type', 'Error'])  # Aggiungi intestazioni
        wb.save(file_path)
    else:
        # Apri il workbook esistente
        wb = openpyxl.load_workbook(file_path)
    return wb

def append_to_xlsx(wb, file_path, data):
    # Carica il foglio di lavoro
    ws = wb.active
    # Aggiungi i nuovi dati
    ws.append(data)
    # Salva il workbook
    wb.save(file_path)

# Percorso del file Excel
filename = '/content/drive/My Drive/Llama2_Translation_LNC/Llama2InteractionCorpus_01_05_IR_EOI_LOI.xlsx'

# Setup iniziale del file Excel
wb = setup_xlsx(filename)

def update_excel(input_text, translation, output, prompt_type, error_type):
    # Dati da aggiungere
    data = [input_text, translation, output, prompt_type, error_type]
    # Aggiorna il file Excel
    append_to_xlsx(wb, filename, data)

# Prompt Analysis

In [ ]:
def extract_between_brackets(input_string):
    # Usa un'espressione regolare per trovare il testo tra < e >
    match = re.search(r'<([^>]*)>', input_string)
    # Se c'è una corrispondenza, restituisci il gruppo catturato, altrimenti restituisci None o una stringa vuota
    if match:
        return match.group(1)
    else:
        return "Output not found!"

In [ ]:
llama2PromptTemplate = lambda userPrompt,systemPrompt: f"""
<s>[INST] <<SYS>>
{systemPrompt}
<</SYS>>

{userPrompt} [/INST]
"""

In [ ]:
systemPrompt="""
You are an assistant expert on finite state automata. Your task is to translate \
user inputs from natural language to a controlled natural language format. \
This translation should adhere to specific vocabulary and syntax guidelines \
suitable for an AIML chatbot. Your response must be precise, using technical \
terminology related to finite state automata where appropriate. Ensure the \
translated text contains clear, unambiguous language that aligns with the \
rules governing the AIML chatbot's response system. Focus on maintaining the \
integrity of the technical content while simplifying the explanation for \
accurate and effective chatbot communication.
"""

In [ ]:
userPromptZeroShot = lambda text: f"""
  Translate the text delimited by triple backticks, without any preambles or \
  additional explanation, into controlled natural language sentence suitable \
  for an AIML system.
  Format the response by directly placing the translation within angle \
  brackets < >.
  For example:
  Output: <Your text here>.
  Now, use the same format for the translation:
  ``` {text} ```
  """

In [ ]:
userPromptChainOfThoughts = lambda text: f"""
Translate '{text}' into controlled natural language for an AIML chatbot, focusing on:
1. Identifying key components related to finite state automata.
2. Using precise, technical terminology where appropriate.
3. Ensuring clarity and adherence to AIML chatbot rules.
Ensure that only the response itself is literally placed between '<' and '>'
"""

In [ ]:
userPromptChainOfThoughts = lambda text: f"""
Translate the user's input considering the following steps:
1. Analyze the user input for key components:
   Input: '{text}'
2. Identify the critical elements and any technical terminology related to finite state automata:
   - Examine keywords and phrases that signify specific automata concepts.
3. Construct a response in controlled natural language:
   - Determine how to express these components clearly and unambiguously.
   - Use controlled language structures that fit the AIML chatbot's rules.
4. Synthesize the translation to ensure clarity, precision, and adherence to technical terms.
5. Finalize the translation ensuring technical accuracy and alignment with AIML standards:
   - Format the response to be enclosed within angular brackets.
6. Explicit instruction for output:
   - Example format for final output: 'Please format the final response like this: <RESPONSE>'
   - Ensure that only the response itself is literally placed between '<' and '>'.
Example:
Input: "Tell me a little bit about the automaton "
Output: "<Describe me briefly the automaton>"
"""

In [ ]:
userPromptZeroChainOfThoughts = lambda text: f"""
  Translate the text delimited by triple backticks, without any preambles or \
  additional explanation, into controlled natural language sentence suitable \
  for an AIML system.
  Format the response by directly placing the translation within angle \
  brackets < >.
  For example:
  Output: <Your text here>.
  Now the same format for the translation:
  ``` {text} ```
  Take a deep breath and work on this problem step-by-step.
  """

## Topic Change

### Zero Shot

In [ ]:
text = "Tell me a little bit about the automaton"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Topic Change")

Input: Tell me a little bit about the automaton


Llama.generate: prefix-match hit


Output: Tell me about the finite state automaton. Please provide information on its syntax, vocabulary, and any relevant technical details.


In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Topic Change")


Input: Talk me about transitions 


Llama.generate: prefix-match hit


Output: Talk me about transitions

Input: is there any self loop


Llama.generate: prefix-match hit


Output:  Does the finite state automaton contain a self-loop? 

Input: What transitions does the automaton have 


...

Output: There exists a transition between state q5 and state q7

Input: Does it only accept 1s and 0s 


Llama.generate: prefix-match hit


Output: Does it only accept 1s and 0s?

Input: what is the initial state of the automaton 


Llama.generate: prefix-match hit


Output: What is the initial state of the finite state automaton?


### One Shot

In [ ]:
userPromptOneShotTC = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "Tell me a little bit about the automaton "
Output: "<Describe me briefly the automaton>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""


In [ ]:
text = "what are the states and the connections between states"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotTC(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Topic Change")

Input: what are the states and the connections between states


Llama.generate: prefix-match hit


Output: Describe the states and the transitions between them


In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotTC(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Topic Change")


Input: Talk me about transitions 


Llama.generate: prefix-match hit


Output: Describe the transitions of the automaton

Input: is there any self loop


Llama.generate: prefix-match hit


Output: Does this automaton have a self-loop?

Input: What transitions does the automaton have 


...

Output: Describe the transition between state q5 and state q7

Input: Does it only accept 1s and 0s 


Llama.generate: prefix-match hit


Output: Is the automaton limited to accepting only 1s and 0s?

Input: what is the initial state of the automaton 


Llama.generate: prefix-match hit


Output: Describe the initial state of the automaton


### Few Shots

Example 1:
Input: "Tell me a little bit about the automaton"
Output: "Describe me briefly the automaton>"

Example 2:
Input: "According to your understanding of Automaton 2..."
Output: "Tell me more about the automaton 2>"

Example 3:
Input: "What is the final state of an automaton"
Output: "Identify the final state of the automaton>"

Example 4:
Input: "How are transitions defined"
Output: "What are the transitions>"

Example 5:
Input: "How is an automaton defined"
Output: "What is an automaton>"

Example 6:
Input: "what are the symbols accepted by the automaton"
Output: "Specify the alphabet used by the automaton.>"

Example 7:
Input: "nodes"
Output: "Tell me more about states>"

Example 8:
Input: "What are the transitions"
Output: "What are transitions>"

Example 9:
Input: "Give me a list of the arcs"
Output: "What are arcs>"

Example 10:
Input: "what is the initial stage of the automaton"
Output: "What is the initial state>"

In [ ]:
userPromptFewShotsTC = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "According to your understanding of Automaton 2..."
Output: "<Tell me more about the automaton 2>"
Example 2:
Input: "What is the final state of an automaton"
Output: "<Identify the final state of the automaton>"
Example 3:
Input: "How are transitions defined"
Output: "<What are the transitions>"
Example 4:
Input: "How is an automaton defined"
Output: "<What is an automaton>"
Example 5:
Input: "what are the symbols accepted by the automaton"
Output: "<Specify the alphabet used by the automaton.>"
Example 6:
Input: "nodes"
Output: "<Tell me more about states>"
Example 7:
Input: "Give me a list of the arcs"
Output: "<What are arcs>"
Example 8:
Input: "what is the initial stage of the automaton"
Output: "<What is the initial state>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "According to your understanding of Automaton 2 what is its optimal spatial representation"
print("\nInput: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsTC(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Topic Change")


Input: According to your understanding of Automaton 2 what is its optimal spatial representation


Llama.generate: prefix-match hit


Output: Tell me more about the optimum spatial representation of Automaton 2


In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsTC(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Topic Change")


Input: Talk me about transitions 


Llama.generate: prefix-match hit


Output: What are the transitions

Input: is there any self loop


Llama.generate: prefix-match hit


Output: Tell me if there are any self-loops in the automaton

Input: What transitions does the automaton have 


...

Output: What are the transitions between states q5 and q7?

Input: Does it only accept 1s and 0s 


Llama.generate: prefix-match hit


Output: Output not found!

Input: what is the initial state of the automaton 


Llama.generate: prefix-match hit


Output: Identify the initial state of the automaton


### Chain of Thoughts

In [ ]:
text = "what are the states and the connections between states"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
print("Completed Output: " + model_output)
#update_excel(text, translation, model_output, "Chain of Thoughts", "Topic Change")

Input: what are the states and the connections between states


Llama.generate: prefix-match hit


Output: state
Completed Output: <state>  A finite state automaton (FSA) consists of a set of states and transitions between them.</state>

These states can be thought of as distinct conditions or situations in which the FSA may reside, and the connections or transitions represent the possible ways for the FSA to move from one state to another based on input received. For example, consider an FSA that recognizes the language {a, b}. The states in this case could be Q0 (the starting state), Q1, and Q2 (the final state). The transitions between them would look like this:

Q0 -> a Q1
Q0 -> b Q2
Q1 -> a Q1
Q1 -> ε Q0 (empty transition)
Q2 -> ε Q0</transition>

This FSA can recognize the language {a, b} by starting in Q0 and shifting to Q1 on input 'a', to Q2 on input 'b', and back to Q0 on input ''.</transition>

By precisely identifying key components of finite state automata and utilizing technical terminology where appropriate, the chatbot can effectively communicate complex ideas with c

In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Topic Change")


Input: Talk me about transitions 


Llama.generate: prefix-match hit


Output: Output not found!

Input: is there any self loop


Llama.generate: prefix-match hit


Output: Output not found!

Input: What transitions does the automaton have 


...

Output: Output not found!

Input: Does it only accept 1s and 0s 


Llama.generate: prefix-match hit


Output: Output not found!

Input: what is the initial state of the automaton 


Llama.generate: prefix-match hit


Output: Output not found!


###Zero Chain of Thoughts

In [ ]:
text = "what are the states and the connections between states"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Topic Change")

Input: what are the states and the connections between states


Llama.generate: prefix-match hit


Output What are the current states and their transitions?


In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Topic Change")


Input: Talk me about transitions 


Llama.generate: prefix-match hit


Output: Talk me about transitions

Input: is there any self loop


Llama.generate: prefix-match hit


Output: Is there any self-loop in the finite state automaton?

Input: What transitions does the automaton have 


...

Output: There is a transition between state q5 and state q7

Input: Does it only accept 1s and 0s 


Llama.generate: prefix-match hit


Output: Does it only accept 1s and 0s?

Input: what is the initial state of the automaton 


Llama.generate: prefix-match hit


Output: What is the initial state of the finite automaton?


## Straight Wrong Response

### Zero Shot

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Straight Wrong Response")

Input: Is there an arc between q0 and q1


Llama.generate: prefix-match hit


Output: Is there a transition from state q0 to state q1?
Update done!


In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Straight Wrong Response")


Input: How many states does a pentagon shaped automaton have 


Llama.generate: prefix-match hit


Output: How many states does a pentagon-shaped automaton have?

Input: arc from q2 to q1


Llama.generate: prefix-match hit


Output: arc-transition from-state="q2" to-state="q1"

Input: how many arcs marked by 1 are there 


...

Output: transitions

Input: Is there an arc between q1 and q2 


Llama.generate: prefix-match hit


Output: Is there an edge/arc connecting state q1 to state q2?

Input: q0


Llama.generate: prefix-match hit


Output: q0


### One Shot

In [ ]:
userPromptOneShotSWR = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "is q4 linked to q0"
Output: "<Is there a transition between q4 and q0>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotSWR(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Straight Wrong Response")

Input: Is there an arc between q0 and q1


Llama.generate: prefix-match hit


Output: Are there any transitions from q0 to q1?


In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotSWR(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Straight Wrong Response")


Input: How many states does a pentagon shaped automaton have 


Llama.generate: prefix-match hit


Output: What is the number of states in a pentagon-shaped finite state machine?

Input: arc from q2 to q1


Llama.generate: prefix-match hit


Output: Does a directed arc exist from state q2 to state q1?

Input: how many arcs marked by 1 are there 


...

Output: Are there any state transitions available in the current FSA?

Input: Is there an arc between q1 and q2 


Llama.generate: prefix-match hit


Output: Does a transition exist between state q1 and state q2?

Input: q0


Llama.generate: prefix-match hit


Output: Is the initial state


### Few Shots

In [ ]:
userPromptFewShotsSWR = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "is q4 linked to q0"
Output: "<Is there a transition between q4 and q0>"
Example 2:
Input: "transitions"
Output: "<What are transitions>"
Example 3:
Input: "How are transitions defined"
Output: "<What are the transitions>"
Example 4:
Input: "arc from q2 to q1"
Output: "<Is there a transition between q2 and q1>"
Example 5:
Input: "q0"
Output: "<What is q0>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsSWR(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Straight Wrong Response")

Llama.generate: prefix-match hit


  Sure! I'd be happy to help you with that. Here's the translation of your input into controlled natural language format:

Input: what are the states and the connections between states

Output: <Specify the states and their transitions>

In
According to your understanding of Automaton 2 what is its optimal spatial representation


In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsSWR(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Straight Wrong Response")


Input: How many states does a pentagon shaped automaton have 


Llama.generate: prefix-match hit


Output: What is the number of states in a pentagonal finite state automaton?

Input: arc from q2 to q1


Llama.generate: prefix-match hit


Output: Is there a transition between q2 and q1

Input: how many arcs marked by 1 are there 


...

Output: What are transitions

Input: Is there an arc between q1 and q2 


Llama.generate: prefix-match hit


Output: Does there exist a transition between q1 and q2

Input: q0


Llama.generate: prefix-match hit


Output: What is q0


### Chain of Thoughts

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Chain of Thoughts", "Straight Wrong Response")

Llama.generate: prefix-match hit


  Sure! I'd be happy to help you translate the user's input into a controlled natural language format suitable for an AIML chatbot. Here's my analysis and translation of the input:

1. Analyze the user input for key components:
Input: "According to your understanding of Automaton 2 what is its optimal spatial representation?"

Keywords and phrases that signify specific automata concepts:

* Automaton 2
* Optimal spatial representation

2. Identify critical elements and technical terminology
what are the states and the connections between states


In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Straight Wrong Response")


Input: How many states does a pentagon shaped automaton have 


Llama.generate: prefix-match hit


Output: Output not found!

Input: arc from q2 to q1


Llama.generate: prefix-match hit


Output: Output not found!

Input: how many arcs marked by 1 are there 


...

Output: Output not found!

Input: Is there an arc between q1 and q2 


Llama.generate: prefix-match hit


Output: Output not found!

Input: q0


Llama.generate: prefix-match hit


Output: Output not found!


###Zero Chain of Thoughts

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Straight Wrong Response")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your input into controlled natural language suitable for an AIML chatbot:

<What are the states and the connections between states?>

In the context of finite state automata, a "state" refers to a specific condition or status that the system can be in. These states are typically represented by nodes or circles in a diagram, and they are connected by arrows or transitions that represent the possible movements or changes between those states. The connections between states are called "edges" or "transitions."

For example, a finite state automaton might have the following states:

* Q0 (the initial state)
* Q1
* Q2

With the following transitions:

* Q0 -> Q1 (when the system receives input "A")
* Q1 -> Q2 (when the system receives input "B")
* Q2 -> Q0 (when the system receives input "C")

In this example, the states are Q0, Q1, and Q2, and the connections between them represent the possible transitions between those states.
what are the states and the 

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Straight Wrong Response")


Input: How many states does a pentagon shaped automaton have 


Llama.generate: prefix-match hit


Output: How many states does a pentagon-shaped automaton have?

Input: arc from q2 to q1


Llama.generate: prefix-match hit


Output: arc from q2 to q1

Input: how many arcs marked by 1 are there 


...

Output: transitions

Input: Is there an arc between q1 and q2 


Llama.generate: prefix-match hit


Output: Is there an arc between q1 and q2?

Input: q0


Llama.generate: prefix-match hit


Output: q0


## Indirect Response

### Zero Shot

In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Indirect Response")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your request into controlled natural language suitable for an AIML chatbot:

<Tell me about the finite state machine.>

This translation adheres to the specific vocabulary and syntax guidelines for an AIML chatbot, using technical terms related to finite state machines where appropriate. The request is clear and unambiguous, ensuring the chatbot can understand and respond accurately.
Tell me a little bit about the automaton


In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Indirect Response")


Input: What happens when a 0 comes to state q1 


Llama.generate: prefix-match hit


Output: What happens when a 0 comes to state q1

Input: The automaton accepts a language allowing words made of an odd number of 0s and 1s 


Llama.generate: prefix-match hit


Output: The automaton accepts a language allowing words composed of an odd number of zeros and ones

Input: is there an arc from q0 to q1 


...

Output: The automaton accepts a language that permits word constructions comprising an odd number of zeros and ones.

Input: is q0 linked to q0 


Llama.generate: prefix-match hit


Output: Is q0 linked to q0?

Input: Is there a transition from q0 to q5 


Llama.generate: prefix-match hit


Output: Is there a transition from state q0 to state q5?


### One Shot

In [ ]:
userPromptOneShotIR = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "is there an arc from q0 to q1"
Output: "<Is there a transition between q4 and q0>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""


In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotIR(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Indirect Response")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your request into controlled natural language suitable for an AIML chatbot:

<Tell me about the finite state machine.>

This translation adheres to the specific vocabulary and syntax guidelines for an AIML chatbot, using technical terms related to finite state machines where appropriate. The request is clear and unambiguous, ensuring the chatbot can understand and respond accurately.
Tell me a little bit about the automaton


In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotIR(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Indirect Response")


Input: What happens when a 0 comes to state q1 


Llama.generate: prefix-match hit


Output: What is the result of a zero symbol (0) entering state q1?

Input: The automaton accepts a language allowing words made of an odd number of 0s and 1s 


Llama.generate: prefix-match hit


Output: The automaton recognizes a language composed of strings with an odd number of 0s and 1s

Input: is there an arc from q0 to q1 


...

Output: Is there a language accepted by the automaton composed of an odd number of zeros and ones?

Input: is q0 linked to q0 


Llama.generate: prefix-match hit


Output: Is there a self-loop in q4?

Input: Is there a transition from q0 to q5 


Llama.generate: prefix-match hit


Output: Is there a transition between q0 and q5


### Few Shots

In [ ]:
userPromptFewShotsIR = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "is there an arc from q0 to q1"
Output: "<Is there a transition between q4 and q0>"
Example 2:
Input: "is q4 a final state"
Output: "<What is q4>"
Example 3:
Input: "The automaton accepts a language allowing words made of an odd number of 0s and 1s"
Output: "<The automaton's alphabet consists of words with an odd number of 0s and 1s>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsIR(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Indirect Response")

Llama.generate: prefix-match hit


  Sure! I'd be happy to help you with that. Here's the translation of your input into controlled natural language format:

Input: what are the states and the connections between states

Output: <Specify the states and their transitions>

In
According to your understanding of Automaton 2 what is its optimal spatial representation


In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsIR(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Indirect Response")


Input: What happens when a 0 comes to state q1 


Llama.generate: prefix-match hit


Output: What is the result of a 0 symbol entering state q1?

Input: The automaton accepts a language allowing words made of an odd number of 0s and 1s 


Llama.generate: prefix-match hit


Output: The automaton recognizes a language composed of words with an odd number of 0s and 1s

Input: is there an arc from q0 to q1 


...

Output: The automaton recognizes a language composed of strings with an odd number of 0s and 1s

Input: is q0 linked to q0 


Llama.generate: prefix-match hit


Output: Is there a self-loop in q0?

Input: Is there a transition from q0 to q5 


Llama.generate: prefix-match hit


Output: Is there a transition between q4 and q5


### Chain of Thoughts

In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Chain of Thoughts", "Indirect Response")

Llama.generate: prefix-match hit


  Sure! I'd be happy to help you translate the user's input into a controlled natural language format suitable for an AIML chatbot. Here's my analysis and translation of the input:

1. Analyze the user input for key components:
Input: "According to your understanding of Automaton 2 what is its optimal spatial representation?"

Keywords and phrases that signify specific automata concepts:

* Automaton 2
* Optimal spatial representation

2. Identify critical elements and technical terminology
what are the states and the connections between states


In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Indirect Response")


Input: What happens when a 0 comes to state q1 


Llama.generate: prefix-match hit


Output: dfa-states-transition-description

Input: The automaton accepts a language allowing words made of an odd number of 0s and 1s 


Llama.generate: prefix-match hit


Output: Output not found!

Input: is there an arc from q0 to q1 


...

Output: Output not found!

Input: is q0 linked to q0 


Llama.generate: prefix-match hit


Output: Output not found!

Input: Is there a transition from q0 to q5 


Llama.generate: prefix-match hit


Output: Output not found!


###Zero Chain of Thoughts

In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Indirect Response")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your input into controlled natural language suitable for an AIML chatbot:

<What are the states and the connections between states?>

In the context of finite state automata, a "state" refers to a specific condition or status that the system can be in. These states are typically represented by nodes or circles in a diagram, and they are connected by arrows or transitions that represent the possible movements or changes between those states. The connections between states are called "edges" or "transitions."

For example, a finite state automaton might have the following states:

* Q0 (the initial state)
* Q1
* Q2

With the following transitions:

* Q0 -> Q1 (when the system receives input "A")
* Q1 -> Q2 (when the system receives input "B")
* Q2 -> Q0 (when the system receives input "C")

In this example, the states are Q0, Q1, and Q2, and the connections between them represent the possible transitions between those states.
what are the states and the 

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Indirect Response")


Input: What happens when a 0 comes to state q1 


Llama.generate: prefix-match hit


Output: What happens when a 0 comes to state q1?

Input: The automaton accepts a language allowing words made of an odd number of 0s and 1s 


Llama.generate: prefix-match hit


Output: The automaton accepts a language allowing words made of an odd number of 0s and 1s

Input: is there an arc from q0 to q1 


...

Output: The automaton accepts a language allowing words made of an odd number of 0s and 1s

Input: is q0 linked to q0 


Llama.generate: prefix-match hit


Output: is q0 linked to q0?

Input: Is there a transition from q0 to q5 


Llama.generate: prefix-match hit


Output: Is there a transition from state q0 to state q5?


## Excess of Information

### Zero Shot

In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Excess of Information")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your request into controlled natural language suitable for an AIML chatbot:

<Tell me about the finite state machine.>

This translation adheres to the specific vocabulary and syntax guidelines for an AIML chatbot, using technical terms related to finite state machines where appropriate. The request is clear and unambiguous, ensuring the chatbot can understand and respond accurately.
Tell me a little bit about the automaton


In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Excess of Information")


Input: How many transitions start from q0 


Llama.generate: prefix-match hit


Output: How many transitions start from state q0?

Input: Please describe the transitions 


Llama.generate: prefix-match hit


Output: Please provide information about the transitions between states.

Input: describe the arcs


Llama.generate: prefix-match hit


Output: Describe the arcs of the finite state machine, including their labels and the transitions they facilitate.

Input: can you describe the transitions


Llama.generate: prefix-match hit


Output: What are the possible state transitions and their conditions?


### One Shot

In [ ]:
userPromptOneShotEOI = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "Please describe the transitions"
Output: "<What are transitions>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""


In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotEOI(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Excess of Information")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your request into controlled natural language suitable for an AIML chatbot:

<Tell me about the finite state machine.>

This translation adheres to the specific vocabulary and syntax guidelines for an AIML chatbot, using technical terms related to finite state machines where appropriate. The request is clear and unambiguous, ensuring the chatbot can understand and respond accurately.
Tell me a little bit about the automaton


In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotEOI(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Excess of Information")


Input: How many transitions start from q0 


Llama.generate: prefix-match hit


Output: What is the number of transitions starting from initial state q0?

Input: Please describe the transitions 


Llama.generate: prefix-match hit


Output: What are transitions

Input: describe the arcs


Llama.generate: prefix-match hit


Output: What are the arcs

Input: can you describe the transitions


Llama.generate: prefix-match hit


Output: What are the transition rules


### Few Shots

In [ ]:
userPromptFewShotsEOI = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "Please describe the transitions"
Output: "<What are transitions>"
Example 2:
Input: "how is marked the arc between q0 and q2"
Output: "<Identify the label on the arc between q0 and q2>"
Example 3:
Input: "which is final state"
Output: "<Tell me more about the final state>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsEOI(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Excess of Information")

Llama.generate: prefix-match hit


  Sure! I'd be happy to help you with that. Here's the translation of your input into controlled natural language format:

Input: what are the states and the connections between states

Output: <Specify the states and their transitions>

In
According to your understanding of Automaton 2 what is its optimal spatial representation


In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsEOI(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Excess of Information")


Input: How many transitions start from q0 


Llama.generate: prefix-match hit


Output: What are the number of transitions starting from q0?

Input: Please describe the transitions 


Llama.generate: prefix-match hit


Output: What are transitions

Input: describe the arcs


Llama.generate: prefix-match hit


Output: Describe the arcs

Input: can you describe the transitions


Llama.generate: prefix-match hit


Output: What are transitions


### Chain of Thoughts

In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Chain of Thoughts", "Excess of Information")

Llama.generate: prefix-match hit


  Sure! I'd be happy to help you translate the user's input into a controlled natural language format suitable for an AIML chatbot. Here's my analysis and translation of the input:

1. Analyze the user input for key components:
Input: "According to your understanding of Automaton 2 what is its optimal spatial representation?"

Keywords and phrases that signify specific automata concepts:

* Automaton 2
* Optimal spatial representation

2. Identify critical elements and technical terminology
what are the states and the connections between states


In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Excess of Information")


Input: How many transitions start from q0 


Llama.generate: prefix-match hit


Output: Output not found!

Input: Please describe the transitions 


Llama.generate: prefix-match hit


Output: Output not found!

Input: describe the arcs


Llama.generate: prefix-match hit


Output: Output not found!

Input: can you describe the transitions


Llama.generate: prefix-match hit


Output: Output not found!


###Zero Chain of Thoughts

In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Excess of Information")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your input into controlled natural language suitable for an AIML chatbot:

<What are the states and the connections between states?>

In the context of finite state automata, a "state" refers to a specific condition or status that the system can be in. These states are typically represented by nodes or circles in a diagram, and they are connected by arrows or transitions that represent the possible movements or changes between those states. The connections between states are called "edges" or "transitions."

For example, a finite state automaton might have the following states:

* Q0 (the initial state)
* Q1
* Q2

With the following transitions:

* Q0 -> Q1 (when the system receives input "A")
* Q1 -> Q2 (when the system receives input "B")
* Q2 -> Q0 (when the system receives input "C")

In this example, the states are Q0, Q1, and Q2, and the connections between them represent the possible transitions between those states.
what are the states and the 

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Excess of Information")


Input: How many transitions start from q0 


Llama.generate: prefix-match hit


Output: How many transitions start from q0?

Input: Please describe the transitions 


Llama.generate: prefix-match hit


Output: Please provide a description of the transitions in the finite state automaton.

Input: describe the arcs


Llama.generate: prefix-match hit


Output: describe the arcs

Input: can you describe the transitions


Llama.generate: prefix-match hit


Output: Can you describe the transitions?


## Lake of Information

### Zero Shot

In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Lake of Information")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your request into controlled natural language suitable for an AIML chatbot:

<Tell me about the finite state machine.>

This translation adheres to the specific vocabulary and syntax guidelines for an AIML chatbot, using technical terms related to finite state machines where appropriate. The request is clear and unambiguous, ensuring the chatbot can understand and respond accurately.
Tell me a little bit about the automaton


In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Lake of Information")


Input: What transitions enter and exit q4 


Llama.generate: prefix-match hit


Output: What transitions enter and exit state Q4?

Input: are there arcs between q2 and q0 


Llama.generate: prefix-match hit


Output: Are there arcs connecting state Q2 to state Q0?

Input: What transitions enter and exit q2 


Llama.generate: prefix-match hit


Output: What transitions enter and exit state q2?

Input: Briefly describe the automaton How many states are there 


Llama.generate: prefix-match hit


Output: The finite state automaton has }briefly describe the automaton< with exactly }three states<.


### One Shot

In [ ]:
userPromptOneShotLOI = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "Briefly describe the automaton How many states are there"
Output: "<Describe me briefly the automaton and tell me how many states are there>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""


In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotLOI(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Lake of Information")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your request into controlled natural language suitable for an AIML chatbot:

<Tell me about the finite state machine.>

This translation adheres to the specific vocabulary and syntax guidelines for an AIML chatbot, using technical terms related to finite state machines where appropriate. The request is clear and unambiguous, ensuring the chatbot can understand and respond accurately.
Tell me a little bit about the automaton


In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotLOI(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Lake of Information")


Input: What transitions enter and exit q4 


Llama.generate: prefix-match hit


Output: Describe to me the transitions that enter and exit state q4

Input: are there arcs between q2 and q0 


Llama.generate: prefix-match hit


Output: Are there transitions from state q2 to state q0?

Input: What transitions enter and exit q2 


Llama.generate: prefix-match hit


Output: Describe the transitions that enter and exit state q2, please?

Input: Briefly describe the automaton How many states are there 


Llama.generate: prefix-match hit


Output: Describe me briefly the automaton and tell me how many states it has


### Few Shots

In [ ]:
userPromptFewShotsLOI = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "Briefly describe the automaton How many states are there"
Output: "<Describe me briefly the automaton and tell me how many states are there>"
Example 2:
Input: "There is a transition between q2 and q0"
Output: "<Is there a transition between q2 and q0>"
Example 3:
Input: "What transitions enter and exit q1"
Output: "<List transitions entering and exiting q1>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsLOI(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Lake of Information")

Llama.generate: prefix-match hit


  Sure! I'd be happy to help you with that. Here's the translation of your input into controlled natural language format:

Input: what are the states and the connections between states

Output: <Specify the states and their transitions>

In
According to your understanding of Automaton 2 what is its optimal spatial representation


In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsLOI(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Lake of Information")


Input: What transitions enter and exit q4 


Llama.generate: prefix-match hit


Output: Describe the transitions entering and exiting q4

Input: are there arcs between q2 and q0 


Llama.generate: prefix-match hit


Output: Are there arcs between q2 and q0?

Input: What transitions enter and exit q2 


Llama.generate: prefix-match hit


Output: Describe me the transitions entering and exiting q2

Input: Briefly describe the automaton How many states are there 


Llama.generate: prefix-match hit


Output: Describe me briefly the automaton and tell me how many states are there


### Chain of Thoughts

In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Chain of Thoughts", "Lake of Information")

Llama.generate: prefix-match hit


  Sure! I'd be happy to help you translate the user's input into a controlled natural language format suitable for an AIML chatbot. Here's my analysis and translation of the input:

1. Analyze the user input for key components:
Input: "According to your understanding of Automaton 2 what is its optimal spatial representation?"

Keywords and phrases that signify specific automata concepts:

* Automaton 2
* Optimal spatial representation

2. Identify critical elements and technical terminology
what are the states and the connections between states


In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Lake of Information")


Input: What transitions enter and exit q4 


Llama.generate: prefix-match hit


Output: Output not found!

Input: are there arcs between q2 and q0 


Llama.generate: prefix-match hit


Output: Output not found!

Input: What transitions enter and exit q2 


Llama.generate: prefix-match hit


Output: ' and '

Input: Briefly describe the automaton How many states are there 


Llama.generate: prefix-match hit


Output: Output not found!


###Zero Chain of Thoughts

In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Lake of Information")

Llama.generate: prefix-match hit


  Sure! Here's the translation of your input into controlled natural language suitable for an AIML chatbot:

<What are the states and the connections between states?>

In the context of finite state automata, a "state" refers to a specific condition or status that the system can be in. These states are typically represented by nodes or circles in a diagram, and they are connected by arrows or transitions that represent the possible movements or changes between those states. The connections between states are called "edges" or "transitions."

For example, a finite state automaton might have the following states:

* Q0 (the initial state)
* Q1
* Q2

With the following transitions:

* Q0 -> Q1 (when the system receives input "A")
* Q1 -> Q2 (when the system receives input "B")
* Q2 -> Q0 (when the system receives input "C")

In this example, the states are Q0, Q1, and Q2, and the connections between them represent the possible transitions between those states.
what are the states and the 

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Lake of Information")


Input: What transitions enter and exit q4 


Llama.generate: prefix-match hit


Output: What transitions enter and exit q4?

Input: are there arcs between q2 and q0 


Llama.generate: prefix-match hit


Output: Are there arcs between state q2 and state q0?

Input: What transitions enter and exit q2 


Llama.generate: prefix-match hit


Output: What transitions enter and exit state q2?

Input: Briefly describe the automaton How many states are there 


Llama.generate: prefix-match hit


Output: Describe the automaton and its number of states, if possible in a single sentence.
